In [1]:
# TODO: fix last dense layer. Is it the number of classes? If so then nothing to fix
# TODO: discuss how reading in dataset could be made multi-threaded (each dataset is read in on own thread then all joined together at the end)

import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import imageio as iio
import os
import xml.etree.ElementTree as ET
import cv2
import numpy as np
from PIL import Image
from numba import cuda  # https://stackoverflow.com/a/52354865/6476994
from keras.utils.np_utils import to_categorical
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import csv
import re
from datetime import datetime
import collections

In [2]:
# allows all images to be displayed at once (else only displays the last call to plt.imshow())
# https://stackoverflow.com/a/41210974
def displayImage(image, caption = None, colour = None) -> None:
    plt.figure()
    if(colour != None):
        plt.imshow(image, cmap=colour)
    else:
        plt.imshow(image)
        
    if(caption != None):
        # display caption below picture (https://stackoverflow.com/a/51486361)
        plt.figtext(0.5, 0.01, caption, wrap=True, horizontalalignment='center', fontsize=12)

In [3]:
# dataset_names = ["BB01", "BB02", "BB03", "BB04", "BB05"]
dataset_names = ["BB01", "BB02", "BB03", "BB04", "BB05", "BB06", "BB07", "BB08", "BB09", "BB10"]
for i in range(11, 37):
    dataset_names.append("BB{}".format(i))
print("dataset_names: {}".format(dataset_names))

dataset_names: ['BB01', 'BB02', 'BB03', 'BB04', 'BB05', 'BB06', 'BB07', 'BB08', 'BB09', 'BB10', 'BB11', 'BB12', 'BB13', 'BB14', 'BB15', 'BB16', 'BB17', 'BB18', 'BB19', 'BB20', 'BB21', 'BB22', 'BB23', 'BB24', 'BB25', 'BB26', 'BB27', 'BB28', 'BB29', 'BB30', 'BB31', 'BB32', 'BB33', 'BB34', 'BB35', 'BB36']


In [4]:
# free up GPU if it didn't after the last run
cuda.select_device(0)
cuda.close()

# Read in dataset

In [5]:
# get the all original output filenames
def readInImages(datasetName):
    print("reading in images for dataset: {}".format(datasetName))
    desired_size = 224
    image_list = []
    imgRegExp = re.compile(r'.*[.](JPG)$')
    # https://stackoverflow.com/a/3207973
    all_image_filenames = next(os.walk('data/{}'.format(datasetName)),
                         (None, None, []))[2]  # [] if no file
    # filter out file names that are not JPEGs
    all_image_filenames = [i for i in all_image_filenames if imgRegExp.match(i)]
    # walk() outputs unordered, so we need to sort
    all_image_filenames.sort()
    # print("all_image_filenames: {}".format(all_image_filenames))
    print("all_image_filenames length: {}".format(len(all_image_filenames)))
    for fn in all_image_filenames:
        # im = Image.open('data/{}/{}'.format(datasetName, fn))
        im = cv2.imread('data/{}/{}'.format(datasetName, fn))
        # resize the image to conserve memory, and transform it to be square while
        # maintaining the aspect ration (give it padding):
        # https://jdhao.github.io/2017/11/06/resize-image-to-square-with-padding/#using-opencv
        old_size = im.shape[:2]
        ratio = float(desired_size)/max(old_size)
        new_size = tuple([int(x*ratio) for x in old_size])
        im = cv2.resize(im, (new_size[1], new_size[0]))

        delta_w = desired_size - new_size[1]
        delta_h = desired_size - new_size[0]
        top, bottom = delta_h//2, delta_h-(delta_h//2)
        left, right = delta_w//2, delta_w-(delta_w//2)

        color = [0, 0, 0]
        new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)

        
        image_list.append(np.asarray(new_im))
    
    print("done current dataset")
    return image_list

all_images = []
for fn in dataset_names:    
    all_images = [*all_images, *readInImages(fn)]

reading in images for dataset: BB01
all_image_filenames length: 285
done current dataset
reading in images for dataset: BB02
all_image_filenames length: 45
done current dataset
reading in images for dataset: BB03
all_image_filenames length: 230
done current dataset
reading in images for dataset: BB04
all_image_filenames length: 999
done current dataset
reading in images for dataset: BB05
all_image_filenames length: 189
done current dataset
reading in images for dataset: BB06
all_image_filenames length: 1137
done current dataset
reading in images for dataset: BB07
all_image_filenames length: 324
done current dataset
reading in images for dataset: BB08
all_image_filenames length: 66
done current dataset
reading in images for dataset: BB09
all_image_filenames length: 357
done current dataset
reading in images for dataset: BB10
all_image_filenames length: 121
done current dataset
reading in images for dataset: BB11
all_image_filenames length: 167
done current dataset
reading in images for 

# Read in dataset's labels

In [6]:
# labels (using dataset's CSV file)

def readInAnnotations(datasetName):
    labelList = []
    # https://realpython.com/python-csv/#reading-csv-files-with-csv
    with open('data/{}/{}.csv'.format(datasetName, datasetName)) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            # print("row: {}".format(row))
            # first row always contains this string, so ignore it
            if "RECONYX - MapView Professional" in row:
                continue
            if line_count == 0:
                # print(f'Column names are {", ".join(row)}')
                line_count += 1
            else:
                # print("Image Name: {}. Hit List: {}".format(row[0], row[22].replace("\n", ", ")))
                # FIXME handle when hitlist contains more than one item (e.g., BB06 IMG_512 has 'kangaroo' and 'empty photo') - sort of handled, need to make more dynamic
                hit_list = row[22]
                if hit_list == '':
                    labelList.append("Empty photo")
                elif hit_list == 'Empty photo\nHuman Presense/Deployment':
                    labelList.append("Human Presense/Deployment")
                elif hit_list == 'Kangaroo\nEmpty photo':
                    labelList.append("Kangaroo")
                else:
                    # FIXME: rendundant case?
                    labelList.append(hit_list.replace("\n", ", "))
                line_count += 1
    # print("returning labelList (length: {}): {}".format(len(labelList), labelList))
    # print("returning labelList of length: {}".format(len(labelList)))
    return labelList

all_image_labels = []
for fn in dataset_names:
    all_image_labels = [*all_image_labels, *readInAnnotations(fn)]

# print("all_image_labels: {}".format(all_image_labels))

classes = set(all_image_labels)
print("all classes (length={}): {}".format(len(classes), classes))

all classes (length=8): {'Other', 'Kangaroo', 'Human Presense/Deployment', 'Cat', 'Empty photo', 'Rabbit', 'Emu', 'Fox'}


# Randomly split the dataset and corresponding labels into training and test sets

In [7]:
print("all_images size: {}".format(len(all_images)))
# print("all_image_labels size: {}".format(len(all_image_labels)))


training_images, test_images, training_labels, test_labels = train_test_split(all_images, all_image_labels, test_size=0.2, random_state=42)

print("training_labels length: {}".format(len(training_labels)))
print("test_labels length: {}".format(len(test_labels)))
# print("test_labels: {}".format(test_labels))
counter = collections.Counter(test_labels)
print("counter: {}".format(counter))

# training_classes = set(training_labels)
training_classes = []
for label in training_labels:
    if label not in training_classes:
        training_classes.append(label)
# test_classes = set(test_labels)
test_classes = []
for label in test_labels:
    if label not in test_classes:
        test_classes.append(label)
print("training_classes (length={}): {}".format(len(training_classes), training_classes))
print("test_classes (length={}): {}".format(len(test_classes), test_classes))

# integer-encode labels so they can be one-hot-encoded
# https://stackoverflow.com/a/56227965/6476994
label_encoder = LabelEncoder()
training_labels = np.array(training_labels)
training_labels = label_encoder.fit_transform(training_labels)
test_labels = np.array(test_labels)
test_labels = label_encoder.fit_transform(test_labels)


# convert list of numpy arrays to numpy array of numpy arrays
# https://stackoverflow.com/a/27516930/6476994
training_images = np.stack(training_images, axis = 0)
test_images = np.stack(test_images, axis = 0)

print("done stacking")
print("training_images shape: {}".format(training_images.shape))
print("test_images shape: {}".format(test_images.shape))

all_images size: 6989
training_labels length: 5591
test_labels length: 1398
counter: Counter({'Kangaroo': 870, 'Empty photo': 227, 'Emu': 119, 'Human Presense/Deployment': 110, 'Fox': 33, 'Cat': 16, 'Other': 16, 'Rabbit': 7})
training_classes (length=8): ['Human Presense/Deployment', 'Empty photo', 'Emu', 'Kangaroo', 'Other', 'Rabbit', 'Fox', 'Cat']
test_classes (length=8): ['Human Presense/Deployment', 'Kangaroo', 'Empty photo', 'Fox', 'Emu', 'Cat', 'Rabbit', 'Other']
done stacking
training_images shape: (5591, 224, 224, 3)
test_images shape: (1398, 224, 224, 3)


# ZFNet

Source: https://towardsdatascience.com/zfnet-an-explanation-of-paper-with-code-f1bd6752121d

## Train the model

In [8]:
# training_images = tf.map_fn(lambda i: tf.stack([i]*3, axis=-1), training_images).numpy()
# test_images = tf.map_fn(lambda i: tf.stack([i]*3, axis=-1), test_images).numpy()

training_images = tf.image.resize(training_images, [224, 224]).numpy()
test_images = tf.image.resize(test_images, [224, 224]).numpy()

training_images = training_images.reshape(training_images.shape)
training_images = training_images / 255.0
test_images = test_images.reshape(test_images.shape)
test_images = test_images / 255.0

training_labels = tf.keras.utils.to_categorical(training_labels, num_classes=len(training_classes))
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=len(test_classes))

num_len_train = int(0.8 * len(training_images))

ttraining_images = training_images[:num_len_train]
ttraining_labels = training_labels[:num_len_train]

valid_images = training_images[num_len_train:]
valid_labels = training_labels[num_len_train:]

training_images = ttraining_images
training_labels = ttraining_labels

model = tf.keras.models.Sequential([
                                    
		tf.keras.layers.Conv2D(96, (7, 7), strides=(2, 2), activation='relu',
			input_shape=(224, 224, 3)),
		tf.keras.layers.MaxPooling2D(3, strides=2),
    tf.keras.layers.Lambda(lambda x: tf.image.per_image_standardization(x)),

		tf.keras.layers.Conv2D(256, (5, 5), strides=(2, 2), activation='relu'),
		tf.keras.layers.MaxPooling2D(3, strides=2),
    tf.keras.layers.Lambda(lambda x: tf.image.per_image_standardization(x)),

		tf.keras.layers.Conv2D(384, (3, 3), activation='relu'),

		tf.keras.layers.Conv2D(384, (3, 3), activation='relu'),

		tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),

		tf.keras.layers.MaxPooling2D(3, strides=2),

    tf.keras.layers.Flatten(),

		tf.keras.layers.Dense(4096),

		tf.keras.layers.Dense(4096),

		tf.keras.layers.Dense(len(classes), activation='softmax')#FIXME is this the number of classes? (check paper)
	])


model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.01, momentum=0.9), \
              loss='categorical_crossentropy', \
              metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(5)])

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', \
                                            		factor=0.1, patience=1, \
																								min_lr=0.00001)
model.fit(training_images, training_labels, batch_size=32, \
          validation_data=(valid_images, valid_labels), \
					epochs=90, callbacks=[reduce_lr])

2022-05-29 09:08:50.963811: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-29 09:08:50.967284: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-29 09:08:50.967474: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-29 09:08:50.967901: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1/90


2022-05-29 09:08:56.959056: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-05-29 09:08:57.274513: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


140/140 [==============================] - 10s 59ms/step - loss: 1.2465 - accuracy: 0.5890 - top_k_categorical_accuracy: 0.9712 - val_loss: 1.0945 - val_accuracy: 0.6032 - val_top_k_categorical_accuracy: 0.9839 - lr: 0.0100
Epoch 2/90
140/140 [==============================] - 7s 50ms/step - loss: 1.0198 - accuracy: 0.6172 - top_k_categorical_accuracy: 0.9897 - val_loss: 0.9513 - val_accuracy: 0.6282 - val_top_k_categorical_accuracy: 0.9875 - lr: 0.0100
Epoch 3/90
140/140 [==============================] - 7s 50ms/step - loss: 0.9486 - accuracy: 0.6391 - top_k_categorical_accuracy: 0.9899 - val_loss: 0.9202 - val_accuracy: 0.6524 - val_top_k_categorical_accuracy: 0.9893 - lr: 0.0100
Epoch 4/90
140/140 [==============================] - 7s 50ms/step - loss: 0.8534 - accuracy: 0.6798 - top_k_categorical_accuracy: 0.9926 - val_loss: 0.8546 - val_accuracy: 0.6738 - val_top_k_categorical_accuracy: 0.9911 - lr: 0.0100
Epoch 5/90
140/140 [==============================] - 7s 50ms/step - loss:

## Evaluate the trained model

In [9]:
print('test_images shape: {}'.format(test_images.shape))
print('test_labels shape: {}'.format(test_labels.shape))

results = model.evaluate(test_images,test_labels)

test_images shape: (1398, 224, 224, 3)
test_labels shape: (1398, 8)
44/44 [==============================] - 1s 21ms/step - loss: 0.4615 - accuracy: 0.8584 - top_k_categorical_accuracy: 0.9971


2022-05-29 09:19:47.945218: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 698.12MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-05-29 09:19:47.945246: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 698.12MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


In [10]:
predictions = (model.predict(test_images) > 0.5).astype("int32")
# print("Predictions (shape: {}):\n{}".format(predictions.shape, predictions))

2022-05-29 09:19:58.190816: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 802.76MiB (rounded to 841752576)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-05-29 09:19:58.190847: I tensorflow/core/common_runtime/bfc_allocator.cc:1027] BFCAllocator dump for GPU_0_bfc
2022-05-29 09:19:58.190858: I tensorflow/core/common_runtime/bfc_allocator.cc:1034] Bin (256): 	Total Chunks: 41, Chunks in use: 41. 10.2KiB allocated for chunks. 10.2KiB in use in bin. 260B client-requested in use in bin.
2022-05-29 09:19:58.190866: I tensorflow/core/common_runtime/bfc_allocator.cc:1034] Bin (512): 	Total Chunks: 2, Chunks in use: 2. 1.0KiB allocated for chunks. 1.0KiB in use in bin. 768B client-requested in use in bin.
2022-05-29 09:19:58.190874: I tens

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
print("test_classes: {}".format(test_classes))
# classification_report uses alphabetic ordering of the classes, so to match the encoded labels to the target_names, provide a sortest list of classes
# https://stackoverflow.com/a/48495303
sorted_test_classes = sorted(test_classes)
print(classification_report(test_labels, predictions, target_names=sorted_test_classes))

# Save the model
* use the current date/time so we can keep incrementation progress of the model as we re-run it

In [ ]:
now = datetime.now()
dt_string = now.strftime('%d-%m-%Y_%H:%M:%S')
print("saving model as: 'ZFNet-{}.h5'.'".format(dt_string))

model.save('ZFNet-{}.h5'.format(dt_string))

## Free up the GPU's memory

In [ ]:
cuda.select_device(0)
cuda.close()